
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [2]:
import os
Root = "/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [23]:
ls

modelForPrediction1.sav  speech-emotion-recognition-ravdess-data/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [6]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [9]:
x_train

array([[-6.21472900e+02,  2.93360786e+01, -1.64034843e+01, ...,
         1.90901192e-05,  1.05907384e-05,  1.02430795e-05],
       [-6.30001465e+02,  5.34919891e+01, -6.52278614e+00, ...,
         8.84829660e-06,  6.52037261e-06,  3.03962543e-06],
       [-6.34491638e+02,  4.20777969e+01, -8.21499729e+00, ...,
         7.87678800e-05,  3.30158036e-05,  3.87367400e-05],
       ...,
       [-6.36160217e+02,  6.34245491e+01,  1.26718636e+01, ...,
         1.68460083e-05,  1.26590767e-05,  1.43330544e-05],
       [-6.43201599e+02,  6.22693329e+01,  2.69707012e+00, ...,
         1.61272674e-05,  1.10278697e-05,  7.53134918e-06],
       [-7.13874939e+02,  5.82035713e+01,  2.24862270e+01, ...,
         5.69948043e-06,  4.37015024e-06,  3.87665887e-06]])

In [10]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [11]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [12]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
#Predict for the test set
y_pred=model.predict(x_test)

In [15]:
y_pred

array(['disgust', 'happy', 'happy', 'fearful', 'fearful', 'happy',
       'fearful', 'fearful', 'disgust', 'fearful', 'disgust', 'disgust',
       'disgust', 'disgust', 'happy', 'fearful', 'calm', 'calm',
       'disgust', 'fearful', 'fearful', 'disgust', 'disgust', 'fearful',
       'fearful', 'disgust', 'calm', 'happy', 'disgust', 'calm', 'calm',
       'happy', 'disgust', 'happy', 'happy', 'disgust', 'calm', 'happy',
       'disgust', 'disgust', 'fearful', 'disgust', 'happy', 'happy',
       'disgust', 'happy', 'happy', 'happy', 'happy', 'disgust',
       'disgust', 'happy', 'disgust', 'disgust', 'happy', 'fearful',
       'fearful', 'happy', 'fearful', 'calm', 'disgust', 'happy', 'happy',
       'disgust', 'happy', 'fearful', 'calm', 'happy', 'happy', 'calm',
       'disgust', 'happy', 'happy', 'fearful', 'fearful', 'disgust',
       'fearful', 'calm', 'calm', 'disgust', 'happy', 'fearful',
       'disgust', 'disgust', 'disgust', 'happy', 'fearful', 'disgust',
       'happy', 'calm

In [16]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.31%


In [17]:
from sklearn.metrics import accuracy_score, f1_score

In [18]:
f1_score(y_test, y_pred,average=None)

array([0.74666667, 0.7107438 , 0.64197531, 0.71028037])

In [19]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,happy,happy
2,happy,happy
3,happy,fearful
4,fearful,fearful
5,happy,happy
6,fearful,fearful
7,calm,fearful
8,disgust,disgust
9,fearful,fearful


In [24]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.pkl', 'wb') as f:
    pickle.dump(model,f)

In [25]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')

In [ ]:
feature